Much of Chicago's murder problem is driven by small fractured gangs, and cycles of retaliation.<sup>1</sup> This is a first attempt of using an LSTM model to predict the likely location of the next murder based on a series of prior murders. While policing alone cannot break these cycles, and no crime-reduction strategy can be complete without investing in historically marginalized communities<sup>2</sup>; this is intended to be a proof of concept for models that could inform intervention-based strategies. This model is not intended for use by Law Enforcement.

1. Hagedorn et al.,"The Fracturing of Gangs and Violence in Chicago: A Research-Based Reorientation of Violence Prevention and Intervention Policy" (UIC Great Cities Institute, Chicago, January, 2019), https://greatcities.uic.edu/wp-content/uploads/2019/01/The_Fracturing_of_Gangs_and_Violence_in_Chicago.pdf
2. US Department of Housing and Urban Development, <i>Neighborhoods and Violent Crime</i> (2016), https://www.huduser.gov/portal/periodicals/em/summer16/highlight2.html

In [63]:
from requests import get
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from arcgis.gis import GIS
from arcgis.geometry import Point, MultiPoint
from arcgis.geocoding import geocode, reverse_geocode
from arcgis.mapping.symbol import create_symbol

In [89]:
df = pd.DataFrame(get('https://data.cityofchicago.org/resource/k9xv-yxzs.json?$limit=10000').json())

In [90]:
#df = df.drop(df[df['domestic']==True].index,axis = 0)
#df = df.drop(df[df['arrest']==True].index,axis = 0)

In [91]:
scaler = StandardScaler()
df['longitude'] = df['longitude'].apply(lambda x: float(x))
df['latitude'] = df['latitude'].apply(lambda x: float(x))
df[['longitude','latitude']] = scaler.fit_transform(df[['longitude','latitude']])

In [171]:
X = []
y = []
n_steps = 4
batch_size = len(df)
for i in range(batch_size - n_steps):
    x = []
    for j in range(i, i + n_steps):
        x_ = [df['latitude'].iloc[j],
              df['longitude'].iloc[j]]
        x.append(x_)
        
    y_ = [df['latitude'].iloc[i + n_steps],
          df['longitude'].iloc[i + n_steps]]
    y.append(y_)
    X.append(x)
        
X = np.array(X)
y = np.array(y)

In [190]:
model = Sequential()
model.add(LSTM(80,input_shape = (X.shape[1],X.shape[2]), return_sequences = True))
model.add(LSTM(80, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(80))

model.add(Dense(y.shape[1]))
model.compile(optimizer = 'adam', loss = 'mse')
model.fit(X[:-10], y[:-10], epochs=300, batch_size=82, shuffle = False)

2023-04-28 02:51:32.520298: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-28 02:51:32.521228: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-28 02:51:32.521852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/300


2023-04-28 02:51:32.748839: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-28 02:51:32.751619: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-28 02:51:32.753394: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

122/122 [==============================] - 5s 9ms/step - loss: 1.0014
Epoch 2/300
122/122 [==============================] - 1s 11ms/step - loss: 1.0009
Epoch 3/300
122/122 [==============================] - 1s 11ms/step - loss: 1.0004
Epoch 4/300
122/122 [==============================] - 1s 7ms/step - loss: 0.9994
Epoch 5/300
122/122 [==============================] - 1s 6ms/step - loss: 0.9988
Epoch 6/300
122/122 [==============================] - 1s 8ms/step - loss: 0.9985
Epoch 7/300
122/122 [==============================] - 1s 11ms/step - loss: 0.9984
Epoch 8/300
122/122 [==============================] - 2s 13ms/step - loss: 0.9983
Epoch 9/300
122/122 [==============================] - 1s 10ms/step - loss: 0.9982
Epoch 10/300
122/122 [==============================] - 2s 16ms/step - loss: 0.9981
Epoch 11/300
122/122 [==============================] - 1s 12ms/step - loss: 0.9979
Epoch 12/300
122/122 [==============================] - 4s 35ms/step - loss: 0.9977
Epoch 13/300
122/

122/122 [==============================] - 1s 9ms/step - loss: 0.1697
Epoch 197/300
122/122 [==============================] - 1s 10ms/step - loss: 0.1602
Epoch 198/300
122/122 [==============================] - 1s 11ms/step - loss: 0.1527
Epoch 199/300
122/122 [==============================] - 1s 9ms/step - loss: 0.1451
Epoch 200/300
122/122 [==============================] - 1s 11ms/step - loss: 0.1392
Epoch 201/300
122/122 [==============================] - 1s 10ms/step - loss: 0.1342
Epoch 202/300
122/122 [==============================] - 1s 10ms/step - loss: 0.1296
Epoch 203/300
122/122 [==============================] - 1s 11ms/step - loss: 0.1258
Epoch 204/300
122/122 [==============================] - 1s 9ms/step - loss: 0.1221
Epoch 205/300
122/122 [==============================] - 1s 9ms/step - loss: 0.1183
Epoch 206/300
122/122 [==============================] - 1s 10ms/step - loss: 0.1152
Epoch 207/300
122/122 [==============================] - 1s 10ms/step - loss: 0.112

In [191]:
preds = scaler.inverse_transform(model.predict(X[-3:]))

2023-04-28 02:58:57.871446: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-28 02:58:57.877778: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-28 02:58:57.880250: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 2s 2s/step


In [192]:
acts = scaler.inverse_transform(y[-3:])

In [193]:
gis = GIS()

In [194]:
pred_sym = create_symbol("point",colors = [0,0,105,100])
act_sym = create_symbol("point",colors = [105,0,0,100])

In [195]:
m = gis.map('Chicago, IL')

In [196]:
for p in preds:
    pt = Point({"x" : float(p[0]), "y" : float(p[1]), 
            "spatialReference" : {"wkid" : 4326}})
    m.draw(pt, symbol = pred_sym)
    
for p in acts:
    pt = Point({"x" : float(p[0]), "y" : float(p[1]), 
            "spatialReference" : {"wkid" : 4326}})
    m.draw(pt, symbol = act_sym)

In [197]:
m

MapView(layout=Layout(height='400px', width='100%'))

This model shows promise but can be further improved. Next steps include feeding more than just the latitude and longitude models to the column, splitting up the data by different parts of the city and training several models, and importing external gang maps of the city and using a tokenization-based approach.